In [1]:
import pandas as pd
import os
os.chdir('../')
from kernel import utils, data_processing, modelling

In [2]:
df = pd.read_parquet('./data/train_transaction.parquet').pipe(utils.detect_id_col)
df = pd.concat([df[df.isFraud == 1], df[df.isFraud == 0].sample(frac=0.1)], axis=0)
df_fea, labels = utils.split_feature_target(df, 'isFraud')
df_fea.head()

Found id col: TransactionID
Set TransactionID as index column


,V334,V324,V312,V130,V128,V127,C11,V276,V249,V20,...,V69,V252,V238,V1,V17,D10,V56,V251,V21,V44
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987203,NaN,NaN,0.0,0.0,0.000000,0.000000,1.0,NaN,NaN,1.0,...,0.0,NaN,NaN,NaN,0.0,80.0,2.0,NaN,0.0,1.0
2987240,NaN,NaN,0.0,0.0,0.000000,0.000000,1.0,0.0,1.0,1.0,...,NaN,1.0,0.0,NaN,1.0,0.0,NaN,1.0,1.0,1.0
2987243,NaN,NaN,0.0,0.0,37.097900,37.097900,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,NaN,1.0,0.0,1.0,1.0,1.0,2.0
2987245,NaN,NaN,0.0,0.0,74.195801,74.195801,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,NaN,1.0,0.0,1.0,1.0,1.0,3.0
2987288,NaN,NaN,0.0,0.0,0.000000,0.000000,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,NaN,1.0,0.0,1.0,1.0,1.0,1.0


In [3]:
cat_cols = [f'card{i}' for i in range(1, 7)] + [f'M{i}' for i in range(1, 10)]
cat_cols += ['P_emaildomain', 'R_emaildomain', 'ProductCD', 'addr1', 'addr2']

In [4]:
dp = data_processing.GenericDataProcessor(df_fea, 'fraud', True, 15, cat_cols=cat_cols)
df_inf = pd.read_csv('./data/test_transaction.csv')
dp.transform(df_inf)

,V334,V324,V312,V130,V128,V127,C11,V276,V249,V20,...,M9_UNK,P_emaildomain,R_emaildomain,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,addr1,addr2
TransactionID,,,,,,,,,,,,,,,,,,,,,
3663549,-0.04751,-0.042065,-0.276586,-0.147750,-0.074966,-0.074266,-0.072864,-0.031142,-0.105518,-1.990365,...,0.0,17,0,0.0,0.0,0.0,0.0,1.0,22,32
3663550,-0.04751,-0.042065,0.180279,0.620335,-0.045579,-0.011713,-0.107016,-0.031142,-0.105518,0.245107,...,1.0,3,0,0.0,0.0,0.0,0.0,1.0,63,32
3663551,-0.04751,-0.042065,-0.276586,2.027085,-0.074966,0.173752,-0.098478,-0.031142,-0.105518,0.245107,...,0.0,20,0,0.0,0.0,0.0,0.0,1.0,126,32
3663552,-0.04751,-0.042065,-0.276586,-0.306464,-0.074966,-0.087192,-0.098478,-0.031142,-0.105518,0.245107,...,1.0,17,0,0.0,0.0,0.0,0.0,1.0,36,32
3663553,-0.04751,-0.042065,0.126582,0.302079,-0.049033,-0.037632,-0.064326,-0.031142,-0.105518,0.245107,...,0.0,17,0,0.0,0.0,0.0,0.0,1.0,53,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170235,-0.04751,-0.042065,-0.276586,-0.306464,-0.074966,-0.087192,-0.107016,-0.031142,-0.105518,0.245107,...,1.0,17,17,1.0,0.0,0.0,0.0,0.0,59,23
4170236,-0.04751,-0.042065,-0.276586,-0.306464,-0.074966,-0.087192,-0.107016,-0.031142,-0.105518,0.245107,...,1.0,20,20,1.0,0.0,0.0,0.0,0.0,150,38
4170237,-0.04751,-0.042065,-0.276586,-0.306464,-0.074966,-0.087192,-0.107016,-0.031142,-0.105518,0.245107,...,0.0,20,0,0.0,0.0,0.0,0.0,1.0,71,32


In [ ]:
clf, params_lr, est_lr = modelling.train_lr_classifier(
    dp.data.values, labels.values, n_iter=30)
lr = est_lr(**params_lr).fit(dp.data.values, labels.values)
df_fea['lr_meta'] = lr.predict_proba(dp.data.values)
lr_meta_train = df_fea.lr_meta.to_dict()
del df_fea
df['lr_meta'] = df.index.map(lr_meta_train).tolist()
df_inf = pd.read_csv('./data/test_transaction.csv')
df_inf['lr_meta'] = lr.predict_proba(dp.transform(df_inf).values)
df['lr_meta'] = df_inf.index.map(lr_meta_train).tolist()

In [ ]:
df_fea, _, _ = data_processing.catboost_preprocessing(df_fea, cat_cols=cat_cols)

In [ ]:
df_inf = pd.read_csv('./data/test_transaction.csv')
df_inf.set_index('TransactionID', inplace=True)
df_inf, _, _ = data_processing.catboost_preprocessing(df_inf, cat_cols=cat_cols)

In [ ]:
params_m = {'iterations':5000,
            'learning_rate':0.05,
            'depth':5,
            'eval_metric':'AUC',
            'verbose':200,
            'od_type':"Iter", # overfit detector
            'od_wait':500, # most recent best iteration to wait before stopping
            'random_seed': 1
            }
cat_model, cr = modelling.train_catboost_classifier(
    df_fea, labels, cat_cols, params=params_m, plot=True)

In [ ]:
df_inf['isFraud'] = cat_model.predict(df_inf[cr])
df_inf = df_inf.reset_index()[['TransactionID', 'isFraud']]
df_inf.to_csv('./data/inf.csv', index=False)

In [ ]:
params = {'iterations':5000,
        'learning_rate':0.01,
        'cat_features':cat_fea,
        'depth':3,
        'eval_metric':'AUC',
        'verbose':200,
        'od_type':"Iter", # overfit detector
        'od_wait':500, # most recent best iteration to wait before stopping
        'random_seed': 1
          }

cat_model = catboost.CatBoostClassifier(**params)
cat_model.fit(X_train, y_train,   
          eval_set=(X_test, y_test), 
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True  
         )

In [ ]:
X_train.head()

In [ ]:
df_inf.head()

In [ ]:
df_inf = pd.read_csv('./data/test_transaction.csv')
df_inf.set_index('TransactionID', inplace=True)
for c in cat_cols:
    df_inf[c] = df_inf[c].astype(str)

In [ ]:
df_inf['isFraud'] = cat_model.predict(df_inf[X_train.columns.tolist()])
df_inf = df_inf.reset_index()[['TransactionID', 'isFraud']]

In [ ]:
df_inf.to_csv('./data/inf.csv', index=False)

In [ ]:
df.shape

In [ ]:
X, y = dp.data.values, labels.values

In [ ]:
clf, params_rf, est_rf = modelling.train_rf_classifier(X, y, n_iter=30)

In [ ]:
from sklearn.ensemble import VotingClassifier
rf = est_rf(**params_rf)
lr = est_lr(**params_lr)
model = VotingClassifier(estimators=[('lr', lr), ('rf', rf)], voting='hard')
model.fit(X, y)

In [ ]:
lr.fit(X, y)

In [ ]:
df_inf = pd.read_csv('./data/test_transaction.csv')
df_inf['isFraud'] = cat_model.predict(dp.transform(df_inf).values)
df_inf = df_inf.reset_index()[['TransactionID', 'isFraud']]
#df_inf.to_csv('./data/inf.csv', index=False)
# df_inf = df_inf.reset_index().rename(columns={'passengerid': 'PassengerId'})
# df_inf[['PassengerId', 'Survived']].to_csv('./data/inf.csv', index=False)

In [ ]:
5500/6347